In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
path = "../dataset/titanic/"

In [3]:
dt_train = pd.read_csv(path+"train.csv")
dt_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
#SibSp is a column that have the numbers of siblins and spouses on boat of someone
#We can see that some people have 0 - 8 others SibSp with them
dt_train['SibSp'].unique()

array([1, 0, 3, 4, 2, 5, 8])

In [5]:
#A way to know who is the SibSp of someone is looking for their surname
matchs = []
for n in dt_train['Name']:
    if "Thayer" in n:
        matchs.append(n)
matchs

['Cumings, Mrs. John Bradley (Florence Briggs Thayer)',
 'Thayer, Mr. John Borland Jr',
 'Thayer, Mrs. John Borland (Marian Longstreth Morris)',
 'Thayer, Mr. John Borland']

In [6]:
# If a woman is married the name of her husband appear before and her name appear in brackets
# Taking a look in the Miss Marian
dt_train[dt_train['Name'] == "Thayer, Mrs. John Borland (Marian Longstreth Morris)"]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
581,582,1,1,"Thayer, Mrs. John Borland (Marian Longstreth M...",female,39.0,1,1,17421,110.8833,C68,C


In [8]:
# Finding her relatives we can see that Miss Marian has a husband and a son with her
dt_train.iloc[[550,581,698]]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
550,551,1,1,"Thayer, Mr. John Borland Jr",male,17.0,0,2,17421,110.8833,C70,C
581,582,1,1,"Thayer, Mrs. John Borland (Marian Longstreth M...",female,39.0,1,1,17421,110.8833,C68,C
698,699,0,1,"Thayer, Mr. John Borland",male,49.0,1,1,17421,110.8833,C68,C


In [ ]:
# df['Age'].fillna(round(dt_train['Age'].mean(), 2), inplace=True)
# df['Fare'].fillna(round(dt_train['Fare'].mean(), 2), inplace=True)
df.fillna(0, inplace=True)

# Coluna que indica se o passageiro é casado ou não
married = np.zeros((len(df['Name']), 1)).astype(int)
for n in df['Name']:
    count = 0
    pos = df[df['Name']==n].index.values.astype(int)[0]
    if "(" in n:
        married[pos] = 1
    else:
        for t in df['Name']:
            if n in t:
                count += 1
        if count > 1:
            married[pos] = 1
df['Married'] = married

# Now, we'll create a new column consider if the spouse of a person is on the boat.
spouses_on_boat = np.zeros((len(df['Name']),1)).astype(int)
for i in range(len(df['SibSp'])):
    s = df.iloc[i]
    if int(s['Married']) and s['SibSp'] > 0:
        spouses_on_boat[i] = 1
df['Spouse_on_boat'] = spouses_on_boat # Saving the new column

# Creating a column that will say if a person has sisters on the boat
sisters = np.zeros((len(df['Name']), 1)).astype(int)
for i in range(len(df['SibSp'])):
    s = df.iloc[i]['SibSp'] - df.iloc[i]['Spouse_on_boat']
    sisters[i] = s
df['Sisters'] = sisters # Saving the new column

passenger_id = df['PassengerId']

df.drop(['PassengerId','Name', 'Cabin', 'Ticket'], axis=1, inplace=True)

le = LabelEncoder()
df['Embarked'] = le.fit_transform(df['Embarked'])
df['Sex'] = le.fit_transform(df['Sex'])

# Scaling the data to use in ml algoritm
scaler_test = StandardScaler()
df_scaled = scaler_test.fit_transform(df)
df_scaled = pd.DataFrame(df_scaled)
df_scaled